# Simple Football Arbitrage Bot

## Brief rundown on how this works:

Different sports bookmakers have different odds on the outcomes of a match (Team A wins, Draw, Team B wins). 

The odds of these match outcomes add up to exactly 100%, but a bookmaker will shift the odds such that the odds of all match otucomes is above 100%, the difference is the bookmaker's edge (i.e. profit). 

For example: PaddyPower rates the odds of Barnsley vs Port Vale (on 2023-07-23) to be 1.62:5:3.5 (win1:win2:draw). Since probabilities are the inverse of the odds we have 0.617:0.20:0.285, this adds up to 1.102, so PaddyPower's edge is 10.2%.

For us to make profit, we have to find odds from different bookmakers that total under 100%. But in reality, since all three outcomes total a 100% probability, we will make money off this difference.

I will be using the-odds-api.com's free API to collect odds from different bookmakers, and then do the rest on here.

In [396]:
import requests
import pandas as pd
import numpy as np
from pandas import json_normalize

In [397]:
with open('API_KEY.txt') as f:
    API_KEY = f.read()
print(f"API key is {API_KEY}")

API key is f6198e12290f5456ce6161b2a5d38720


The next block is straight from the-odds-api.com's sample code. No need to make it myself.

In [398]:
SPORT = 'soccer_england_league1' # use the sport_key from the /sports endpoint below, or use 'upcoming' to see the next 8 games across all sports

REGIONS = 'uk' # uk | us | eu | au. Multiple can be specified if comma delimited

MARKETS = 'h2h' # h2h | spreads | totals. Multiple can be specified if comma delimited

ODDS_FORMAT = 'decimal' # decimal | american

DATE_FORMAT = 'iso' # iso | unix


# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
#
# Now get a list of live & upcoming games for the sport you want, along with odds for different bookmakers
# This will deduct from the usage quota
# The usage quota cost = [number of markets specified] x [number of regions specified]
# For examples of usage quota costs, see https://the-odds-api.com/liveapi/guides/v4/#usage-quota-costs
#
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 

odds_response = requests.get(
    f'https://api.the-odds-api.com/v4/sports/{SPORT}/odds',
    params={
        'api_key': API_KEY,
        'regions': REGIONS,
        'markets': MARKETS,
        'oddsFormat': ODDS_FORMAT,
        'dateFormat': DATE_FORMAT,
    }
)

if odds_response.status_code != 200:
    print(f'Failed to get odds: status_code {odds_response.status_code}, response body {odds_response.text}')

else:
    odds_json = odds_response.json()
    print('Number of events:', len(odds_json))
    print(odds_json)

    # Check the usage quota
    print('Remaining requests', odds_response.headers['x-requests-remaining'])
    print('Used requests', odds_response.headers['x-requests-used'])

Number of events: 12
[{'id': '8372d6cc49af461e9a719f8b63f74055', 'sport_key': 'soccer_england_league1', 'sport_title': 'League 1', 'commence_time': '2023-08-05T14:00:00Z', 'home_team': 'Barnsley', 'away_team': 'Port Vale', 'bookmakers': [{'key': 'boylesports', 'title': 'BoyleSports', 'last_update': '2023-07-23T21:18:17Z', 'markets': [{'key': 'h2h', 'last_update': '2023-07-23T21:18:17Z', 'outcomes': [{'name': 'Barnsley', 'price': 1.7}, {'name': 'Port Vale', 'price': 4.75}, {'name': 'Draw', 'price': 3.3}]}]}, {'key': 'paddypower', 'title': 'Paddy Power', 'last_update': '2023-07-23T21:18:51Z', 'markets': [{'key': 'h2h', 'last_update': '2023-07-23T21:18:51Z', 'outcomes': [{'name': 'Barnsley', 'price': 1.62}, {'name': 'Port Vale', 'price': 5.0}, {'name': 'Draw', 'price': 3.5}]}]}, {'key': 'skybet', 'title': 'Sky Bet', 'last_update': '2023-07-23T21:18:48Z', 'markets': [{'key': 'h2h', 'last_update': '2023-07-23T21:18:48Z', 'outcomes': [{'name': 'Barnsley', 'price': 1.7}, {'name': 'Port Vale',

Now we'll make odds_json more readable using pandas.

First we have to make a function that reads odds_json into something that pandas can handle.

In [399]:
def extract_info(item):
    """
    Extracts match and bookmaker information from a given dictionary representing a single match.

    Parameters:
        item: A dictionary containing information about a specific match.
        
        The dictionary will contain the following keys:
                     - 'home_team': The name of the home team.
                     - 'away_team': The name of the away team.
                     - 'commence_time': The date and time when the match starts (in ISO 8601 format).
                     - 'bookmakers': A list of dictionaries, each representing a bookmaker offering odds for the match.
                       Each bookmaker dictionary should have the following keys:
                       - 'title': The name of the bookmaker.
                       - 'last_update': The date and time when the bookmaker's odds were last updated (in ISO 8601 format).
                       - 'markets': A list of dictionaries, each representing a market for the match (e.g., 'h2h' for
                         head-to-head market).
                         Each market dictionary should have the following keys:
                         - 'outcomes': A list of dictionaries, each representing an outcome in the market.
                           Each outcome dictionary should have the following keys:
                           - 'name': The name of the outcome (e.g., team name or 'Draw' for a draw).
                           - 'price': The odds value for the corresponding outcome.

    Returns:
        extracted_data: A list of dictionaries, each with information for a specific bookmaker and market outcome.
              Each dictionary has the following keys:
              - 'team 1 name': The name of the home team.
              - 'team 2 name': The name of the away team.
              - 'date': The date and time when the match starts (in ISO 8601 format).
              - 'bookmaker name': The name of the bookmaker.
              - 'last bookmaker update': The date and time when the bookmaker's odds were last updated (in ISO 8601 format).
              - 'team 1 odds': The odds value for the home team.
              - 'team 2 odds': The odds value for the away team.
              - 'Draw odds': The odds value for a draw outcome.
    """
    extracted_data = []
    for bookmaker in item['bookmakers']: # We iterate over bookmakers and markets because they are nested in the json
        for market in bookmaker['markets']:
            outcomes = {outcome['name']: outcome['price'] for outcome in market['outcomes']}
            extracted_data.append({
                'Team 1': item['home_team'],
                'Team 2': item['away_team'],
                'Date': item['commence_time'],
                'Bookmaker': bookmaker['title'],
                'Team 1 win odds': outcomes.get(item['home_team'], None),
                'Team 2 win odds': outcomes.get(item['away_team'], None),
                'Draw odds': outcomes.get('Draw', None),
            })
    return extracted_data


In [400]:
data = odds_json

match_tables = {} # Dictionary that stores the DataFrames for every match

for item in data:
    match_name = f"{item['home_team']} vs {item['away_team']}"
    match_data = extract_info(item) 
    if match_name not in match_tables:
        match_tables[match_name] = pd.DataFrame(match_data)
    else:
        match_tables[match_name] = pd.concat([match_tables[match_name], pd.DataFrame(match_data)])
        
# if multiple occurrences of the same match are encountered, their data is concatenated

# Display each match's DataFrame
for match_name, match_df in match_tables.items():
    print(f"Table for {match_name}:")
    display(match_df)
    print("\n")

Table for Barnsley vs Port Vale:


Team 1     Team 2                  Date      Bookmaker  Team 1 win odds  \
0   Barnsley  Port Vale  2023-08-05T14:00:00Z    BoyleSports             1.70   
1   Barnsley  Port Vale  2023-08-05T14:00:00Z    Paddy Power             1.62   
2   Barnsley  Port Vale  2023-08-05T14:00:00Z        Sky Bet             1.70   
3   Barnsley  Port Vale  2023-08-05T14:00:00Z     Bet Victor             1.70   
4   Barnsley  Port Vale  2023-08-05T14:00:00Z          Coral             1.80   
5   Barnsley  Port Vale  2023-08-05T14:00:00Z      Ladbrokes             1.80   
6   Barnsley  Port Vale  2023-08-05T14:00:00Z       888sport             1.73   
7   Barnsley  Port Vale  2023-08-05T14:00:00Z         Betway             1.70   
8   Barnsley  Port Vale  2023-08-05T14:00:00Z        Betfair             1.68   
9   Barnsley  Port Vale  2023-08-05T14:00:00Z        Betfair             2.04   
10  Barnsley  Port Vale  2023-08-05T14:00:00Z     Virgin Bet             1.67   
11  Barnsley  Port Vale  2023-08-05T14:00:00Z  LiveScore Bet             1.67   
12  Barnsley  Port Vale  2023-08-05T14:00:00Z       Mr Green             1.70   
13  Barnsley  Port Vale  2023-08-05T14:00:00Z         Casumo             1.70   
14  Barnsley  Port Vale  2023-08-05T14:00:00Z         Unibet             1.70   
15  Barnsley  Port Vale  2023-08-05T14:00:00Z       LeoVegas             1.68   
16  Barnsley  Port Vale  2023-08-05T14:00:00Z   William Hill             1.75   

    Team 2 win odds  Draw odds  
0              4.75       3.30  
1              5.00       3.50  
2              4.50       3.50  
3              4.33       3.50  
4              4.60       3.40  
5              4.60       3.40  
6              4.50       3.40  
7              4.20       3.40  
8              4.40       3.20  
9              6.40       4.40  
10             4.20       3.40  
11             4.20       3.40  
12             4.25       3.35  
13             4.50       3.50  
14             4.50       3.50  
15             4.20       3.35  
16             4.33       3.70



Table for Blackpool vs Burton Albion:


Team 1         Team 2                  Date      Bookmaker  \
0   Blackpool  Burton Albion  2023-08-05T14:00:00Z    BoyleSports   
1   Blackpool  Burton Albion  2023-08-05T14:00:00Z    Paddy Power   
2   Blackpool  Burton Albion  2023-08-05T14:00:00Z        Sky Bet   
3   Blackpool  Burton Albion  2023-08-05T14:00:00Z     Bet Victor   
4   Blackpool  Burton Albion  2023-08-05T14:00:00Z          Coral   
5   Blackpool  Burton Albion  2023-08-05T14:00:00Z      Ladbrokes   
6   Blackpool  Burton Albion  2023-08-05T14:00:00Z       888sport   
7   Blackpool  Burton Albion  2023-08-05T14:00:00Z         Betway   
8   Blackpool  Burton Albion  2023-08-05T14:00:00Z        Betfair   
9   Blackpool  Burton Albion  2023-08-05T14:00:00Z        Betfair   
10  Blackpool  Burton Albion  2023-08-05T14:00:00Z     Virgin Bet   
11  Blackpool  Burton Albion  2023-08-05T14:00:00Z  LiveScore Bet   
12  Blackpool  Burton Albion  2023-08-05T14:00:00Z       Mr Green   
13  Blackpool  Burton Albion  2023-08-05T14:00:00Z         Casumo   
14  Blackpool  Burton Albion  2023-08-05T14:00:00Z         Unibet   
15  Blackpool  Burton Albion  2023-08-05T14:00:00Z       LeoVegas   
16  Blackpool  Burton Albion  2023-08-05T14:00:00Z   William Hill   

    Team 1 win odds  Team 2 win odds  Draw odds  
0              1.55             5.00       3.80  
1              1.53             5.00       4.00  
2              1.53             5.25       4.00  
3              1.55             4.80       4.00  
4              1.67             5.20       3.90  
5              1.67             5.20       3.90  
6              1.57             5.50       3.80  
7              1.55             4.75       3.80  
8              1.55             4.90       4.20  
9              1.76             6.20       5.20  
10             1.48             4.80       4.00  
11             1.48             4.80       4.00  
12             1.55             4.70       3.85  
13             1.55             5.00       4.05  
14             1.55             5.00       4.05  
15             1.54             4.60       3.80  
16             1.55             5.50       4.20



Table for Bolton vs Lincoln City:


...

Team 1       Team 2                  Date      Bookmaker  \
0   Wycombe Wanderers  Exeter City  2023-08-05T14:00:00Z    BoyleSports   
1   Wycombe Wanderers  Exeter City  2023-08-05T14:00:00Z    Paddy Power   
2   Wycombe Wanderers  Exeter City  2023-08-05T14:00:00Z        Sky Bet   
3   Wycombe Wanderers  Exeter City  2023-08-05T14:00:00Z     Bet Victor   
4   Wycombe Wanderers  Exeter City  2023-08-05T14:00:00Z          Coral   
5   Wycombe Wanderers  Exeter City  2023-08-05T14:00:00Z      Ladbrokes   
6   Wycombe Wanderers  Exeter City  2023-08-05T14:00:00Z       888sport   
7   Wycombe Wanderers  Exeter City  2023-08-05T14:00:00Z         Betway   
8   Wycombe Wanderers  Exeter City  2023-08-05T14:00:00Z        Betfair   
9   Wycombe Wanderers  Exeter City  2023-08-05T14:00:00Z        Betfair   
10  Wycombe Wanderers  Exeter City  2023-08-05T14:00:00Z     Virgin Bet   
11  Wycombe Wanderers  Exeter City  2023-08-05T14:00:00Z  LiveScore Bet   
12  Wycombe Wanderers  Exeter City  2023-08-05T14:00:00Z       Mr Green   
13  Wycombe Wanderers  Exeter City  2023-08-05T14:00:00Z         Casumo   
14  Wycombe Wanderers  Exeter City  2023-08-05T14:00:00Z         Unibet   
15  Wycombe Wanderers  Exeter City  2023-08-05T14:00:00Z       LeoVegas   
16  Wycombe Wanderers  Exeter City  2023-08-05T14:00:00Z   William Hill   

    Team 1 win odds  Team 2 win odds  Draw odds  
0              1.83             4.00       3.20  
1              1.91             3.70       3.40  
2              1.85             3.60       3.50  
3              1.87             3.60       3.50  
4              1.95             3.90       3.50  
5              1.91             3.90       3.50  
6              1.91             3.75       3.40  
7              1.85             3.50       3.40  
8              1.88             3.90       3.55  
9              2.10             4.70       4.20  
10             1.80             3.50       3.40  
11             1.80             3.50       3.40  
12             1.83             3.65       3.40  
13             1.85             3.80       3.55  
14             1.85             3.80       3.55  
15             1.82             3.60       3.40  
16             1.91             3.75       3.60



Table for Northampton Town vs Stevenage:


Team 1     Team 2                  Date      Bookmaker  \
0   Northampton Town  Stevenage  2023-08-05T14:00:00Z         Betway   
1   Northampton Town  Stevenage  2023-08-05T14:00:00Z    Paddy Power   
2   Northampton Town  Stevenage  2023-08-05T14:00:00Z       888sport   
3   Northampton Town  Stevenage  2023-08-05T14:00:00Z    BoyleSports   
4   Northampton Town  Stevenage  2023-08-05T14:00:00Z      Ladbrokes   
5   Northampton Town  Stevenage  2023-08-05T14:00:00Z          Coral   
6   Northampton Town  Stevenage  2023-08-05T14:00:00Z        Sky Bet   
7   Northampton Town  Stevenage  2023-08-05T14:00:00Z        Betfair   
8   Northampton Town  Stevenage  2023-08-05T14:00:00Z        Betfair   
9   Northampton Town  Stevenage  2023-08-05T14:00:00Z     Virgin Bet   
10  Northampton Town  Stevenage  2023-08-05T14:00:00Z  LiveScore Bet   
11  Northampton Town  Stevenage  2023-08-05T14:00:00Z       Mr Green   
12  Northampton Town  Stevenage  2023-08-05T14:00:00Z         Casumo   
13  Northampton Town  Stevenage  2023-08-05T14:00:00Z         Unibet   
14  Northampton Town  Stevenage  2023-08-05T14:00:00Z       LeoVegas   
15  Northampton Town  Stevenage  2023-08-05T14:00:00Z   William Hill   

    Team 1 win odds  Team 2 win odds  Draw odds  
0              2.60             2.50       2.88  
1              2.62             2.60       3.10  
2              2.62             2.60       3.20  
3              2.62             2.60       3.00  
4              2.75             2.75       3.00  
5              2.75             2.75       3.00  
6              2.62             2.62       3.00  
7              2.56             2.78       2.82  
8              3.40             3.20       3.35  
9              2.55             2.55       2.88  
10             2.55             2.55       2.88  
11             2.55             2.60       3.00  
12             2.60             2.65       3.10  
13             2.60             2.65       3.10  
14             2.50             2.55       3.00  
15             2.62             2.70       3.10



Table for Reading vs Peterborough United:


Team 1               Team 2                  Date      Bookmaker  \
0   Reading  Peterborough United  2023-08-05T14:00:00Z         Betway   
1   Reading  Peterborough United  2023-08-05T14:00:00Z    Paddy Power   
2   Reading  Peterborough United  2023-08-05T14:00:00Z       888sport   
3   Reading  Peterborough United  2023-08-05T14:00:00Z     Bet Victor   
4   Reading  Peterborough United  2023-08-05T14:00:00Z    BoyleSports   
5   Reading  Peterborough United  2023-08-05T14:00:00Z      Ladbrokes   
6   Reading  Peterborough United  2023-08-05T14:00:00Z          Coral   
7   Reading  Peterborough United  2023-08-05T14:00:00Z        Sky Bet   
8   Reading  Peterborough United  2023-08-05T14:00:00Z        Betfair   
9   Reading  Peterborough United  2023-08-05T14:00:00Z        Betfair   
10  Reading  Peterborough United  2023-08-05T14:00:00Z     Virgin Bet   
11  Reading  Peterborough United  2023-08-05T14:00:00Z  LiveScore Bet   
12  Reading  Peterborough United  2023-08-05T14:00:00Z       Mr Green   
13  Reading  Peterborough United  2023-08-05T14:00:00Z         Casumo   
14  Reading  Peterborough United  2023-08-05T14:00:00Z         Unibet   
15  Reading  Peterborough United  2023-08-05T14:00:00Z       LeoVegas   
16  Reading  Peterborough United  2023-08-05T14:00:00Z   William Hill   

    Team 1 win odds  Team 2 win odds  Draw odds  
0              2.70             2.40       3.00  
1              2.62             2.40       3.30  
2              2.62             2.50       3.25  
3              2.62             2.45       3.25  
4              2.60             2.50       3.25  
5              2.87             2.45       3.25  
6              2.87             2.45       3.25  
7              2.70             2.40       3.20  
8              2.74             2.34       3.40  
9              3.30             2.70       4.20  
10             2.55             2.38       3.10  
11             2.55             2.38       3.10  
12             2.65             2.40       3.20  
13             2.70             2.45       3.30  
14             2.70             2.45       3.30  
15             2.63             2.38       3.15  
16             2.70             2.45       3.40

Let's analyse the first game

In [401]:
df = match_tables['Barnsley vs Port Vale']

df['Team 1 win prob'] = 1 / df['Team 1 win odds']
df['Team 2 win prob'] = 1 / df['Team 2 win odds']
df['Draw prob'] = 1 / df['Draw odds']

display(df)

Team 1     Team 2                  Date      Bookmaker  Team 1 win odds  \
0   Barnsley  Port Vale  2023-08-05T14:00:00Z    BoyleSports             1.70   
1   Barnsley  Port Vale  2023-08-05T14:00:00Z    Paddy Power             1.62   
2   Barnsley  Port Vale  2023-08-05T14:00:00Z        Sky Bet             1.70   
3   Barnsley  Port Vale  2023-08-05T14:00:00Z     Bet Victor             1.70   
4   Barnsley  Port Vale  2023-08-05T14:00:00Z          Coral             1.80   
5   Barnsley  Port Vale  2023-08-05T14:00:00Z      Ladbrokes             1.80   
6   Barnsley  Port Vale  2023-08-05T14:00:00Z       888sport             1.73   
7   Barnsley  Port Vale  2023-08-05T14:00:00Z         Betway             1.70   
8   Barnsley  Port Vale  2023-08-05T14:00:00Z        Betfair             1.68   
9   Barnsley  Port Vale  2023-08-05T14:00:00Z        Betfair             2.04   
10  Barnsley  Port Vale  2023-08-05T14:00:00Z     Virgin Bet             1.67   
11  Barnsley  Port Vale  2023-08-05T14:00:00Z  LiveScore Bet             1.67   
12  Barnsley  Port Vale  2023-08-05T14:00:00Z       Mr Green             1.70   
13  Barnsley  Port Vale  2023-08-05T14:00:00Z         Casumo             1.70   
14  Barnsley  Port Vale  2023-08-05T14:00:00Z         Unibet             1.70   
15  Barnsley  Port Vale  2023-08-05T14:00:00Z       LeoVegas             1.68   
16  Barnsley  Port Vale  2023-08-05T14:00:00Z   William Hill             1.75   

    Team 2 win odds  Draw odds  Team 1 win prob  Team 2 win prob  Draw prob  
0              4.75       3.30         0.588235         0.210526   0.303030  
1              5.00       3.50         0.617284         0.200000   0.285714  
2              4.50       3.50         0.588235         0.222222   0.285714  
3              4.33       3.50         0.588235         0.230947   0.285714  
4              4.60       3.40         0.555556         0.217391   0.294118  
5              4.60       3.40         0.555556         0.217391   0.294118  
6              4.50       3.40         0.578035         0.222222   0.294118  
7              4.20       3.40         0.588235         0.238095   0.294118  
8              4.40       3.20         0.595238         0.227273   0.312500  
9              6.40       4.40         0.490196         0.156250   0.227273  
10             4.20       3.40         0.598802         0.238095   0.294118  
11             4.20       3.40         0.598802         0.238095   0.294118  
12             4.25       3.35         0.588235         0.235294   0.298507  
13             4.50       3.50         0.588235         0.222222   0.285714  
14             4.50       3.50         0.588235         0.222222   0.285714  
15             4.20       3.35         0.595238         0.238095   0.298507  
16             4.33       3.70         0.571429         0.230947   0.270270

In [402]:
import collections

team_1_win_list = list(df['Team 1 win odds'])
team_2_win_list = list(df['Team 2 win odds'])
draw_list = list(df['Draw odds'])

team_1_win_dict = {}
team_2_win_dict = {}
draw_dict = {}

df['Bookmaker'].iloc[0]

for i in range(len(team_1_win_list)):
    team_1_win_dict[df['Bookmaker'].iloc[i]] = team_1_win_list[i]
    team_2_win_dict[df['Bookmaker'].iloc[i]] = team_2_win_list[i]
    draw_dict[df['Bookmaker'].iloc[i]] = draw_list[i]

unique_triplets = collections.defaultdict(list) # {('BoyleSports', 'Paddy Power', 'Sky Bet') : (1.70, 4.75, 3.75)}
    
for key1, val1 in team_1_win_dict.items():
    for key2, val2 in team_2_win_dict.items():
        for key3, val3 in draw_dict.items():

            if key1 != key2 and key1 != key3 and key2 != key3:
                string = key1 + " " + key2 + " " + key3
                unique_triplets[string] = [val1, val2, val3]
            
    

In [403]:
arbitrage_opportunities = {}

for key, val in unique_triplets.items():
    team1win, team2win, draw = val
    if 1/ team1win + 1 / team2win + 1 / draw < 1:
        arbitrage_opportunities[key] = val
        


In [409]:
arbitrage_df = pd.DataFrame.from_dict(arbitrage_opportunities).T

arbitrage_df['Profit (%)'] = (1 - (1 / arbitrage_df[0] + 1 / arbitrage_df[1] + 1 / arbitrage_df[2])) * 100

arbitrage_df.rename(columns={0: "Team 1 win odds", 1: "Team 2 win odds", 2 : "Draw odds"}, inplace = True)

arbitrage_df.sort_values(by = 'Profit (%)', ascending = False, inplace = True)

display(arbitrage_df)

Team 1 win odds  Team 2 win odds  \
Betfair Paddy Power William Hill               2.04             5.00   
Betfair BoyleSports William Hill               2.04             4.75   
Betfair Paddy Power Casumo                     2.04             5.00   
Betfair Paddy Power Bet Victor                 2.04             5.00   
Betfair Paddy Power Sky Bet                    2.04             5.00   
...                                             ...              ...   
Betfair LiveScore Bet William Hill             2.04             4.20   
Betfair LeoVegas William Hill                  2.04             4.20   
William Hill Paddy Power Betfair               1.75             5.00   
Betfair BoyleSports Mr Green                   2.04             4.75   
Betfair BoyleSports LeoVegas                   2.04             4.75   

                                    Draw odds  Profit (%)  
Betfair Paddy Power William Hill         3.70    3.953365  
Betfair BoyleSports William Hill         3.70    2.900734  
Betfair Paddy Power Casumo               3.50    2.408964  
Betfair Paddy Power Bet Victor           3.50    2.408964  
Betfair Paddy Power Sky Bet              3.50    2.408964  
...                                       ...         ...  
Betfair LiveScore Bet William Hill       3.70    0.143841  
Betfair LeoVegas William Hill            3.70    0.143841  
William Hill Paddy Power Betfair         4.40    0.129870  
Betfair BoyleSports Mr Green             3.35    0.077014  
Betfair BoyleSports LeoVegas             3.35    0.077014  

[84 rows x 4 columns]